In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data.sampler import BatchSampler

In [2]:
device = torch.device('cpu')

In [3]:
input_size = 784 # 28x28
hidden_size = 500 
num_classes = 10
num_epochs = 2
batch_size = 100
#test_batch_size = 10000
learning_rate = 0.001
spc=5000
#SPC is for number of samples per class
t1=[]
t2=[]
t3=[]
i=0

In [4]:
# MNIST dataset 

train_dataset = torchvision.datasets.FashionMNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))]),  
                                           download=True)

test_dataset = torchvision.datasets.FashionMNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))]))


for i in range (10):
    t0=(train_dataset.targets==i).nonzero(as_tuple=False)
    t0 = t0.reshape(1, -1)
    t0 = t0.squeeze()
    t0=t0.tolist()
    np.random.shuffle(t0)
    t0=t0[0:spc]
    t1=t1+t0
    print ( "Current list size is" ,len(t1))
    print (spc," random samples for class" ,i, "is added to the list")

train_subset = torch.utils.data.Subset(train_dataset,t1)
#train_dataset0.targets = train_dataset.targets[t1]
#train_dataset0.data = train_dataset.data[t1]

t0=test_dataset.targets
t0 = t0.reshape(1, -1)
t0 = t0.squeeze()
t2=t0.tolist()
print ( "Test list size is" ,len(t2))

Current list size is 5000
5000  random samples for class 0 is added to the list
Current list size is 10000
5000  random samples for class 1 is added to the list
Current list size is 15000
5000  random samples for class 2 is added to the list
Current list size is 20000
5000  random samples for class 3 is added to the list
Current list size is 25000
5000  random samples for class 4 is added to the list
Current list size is 30000
5000  random samples for class 5 is added to the list
Current list size is 35000
5000  random samples for class 6 is added to the list
Current list size is 40000
5000  random samples for class 7 is added to the list
Current list size is 45000
5000  random samples for class 8 is added to the list
Current list size is 50000
5000  random samples for class 9 is added to the list
Test list size is 10000


In [5]:
# Data loader


train_loader = DataLoader(dataset=train_subset, batch_size=batch_size, shuffle=False)


test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size ,shuffle=False)

print (len(test_loader))
print (len(test_dataset))



100
10000


In [6]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = torch.nn.Linear(input_size, hidden_size) 
        self.relu = torch.nn.ReLU()
        self.l2 = torch.nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        # no activation and no softmax at the end
        return out

In [7]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [8]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  


In [9]:
# Train the model for all elements of the dataset
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # origin shape: [100, 1, 28, 28]
        # resized: [100, 784]
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')


Epoch [1/2], Step [100/500], Loss: 0.0000
Epoch [1/2], Step [200/500], Loss: 0.0010
Epoch [1/2], Step [300/500], Loss: 0.0000
Epoch [1/2], Step [400/500], Loss: 0.0000
Epoch [1/2], Step [500/500], Loss: 0.0000
Epoch [2/2], Step [100/500], Loss: 0.0001
Epoch [2/2], Step [200/500], Loss: 0.0111
Epoch [2/2], Step [300/500], Loss: 0.0000
Epoch [2/2], Step [400/500], Loss: 0.0000
Epoch [2/2], Step [500/500], Loss: 0.0001


In [10]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
t3=[]
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        tpredict=predicted.tolist()
        t3=t3+tpredict
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        print("Current total samples are",n_samples,)
        print("of which ",n_correct,"samples are correct")

        
print(len(t3))
       
acc = 100.0 * n_correct / n_samples
print(f'Accuracy of the network on the given test images: {acc} %')


Current total samples are 100
of which  8 samples are correct
Current total samples are 200
of which  21 samples are correct
Current total samples are 300
of which  33 samples are correct
Current total samples are 400
of which  46 samples are correct
Current total samples are 500
of which  62 samples are correct
Current total samples are 600
of which  74 samples are correct
Current total samples are 700
of which  86 samples are correct
Current total samples are 800
of which  98 samples are correct
Current total samples are 900
of which  109 samples are correct
Current total samples are 1000
of which  118 samples are correct
Current total samples are 1100
of which  134 samples are correct
Current total samples are 1200
of which  145 samples are correct
Current total samples are 1300
of which  157 samples are correct
Current total samples are 1400
of which  167 samples are correct
Current total samples are 1500
of which  181 samples are correct
Current total samples are 1600
of which  19

In [11]:
from sklearn.metrics import classification_report
#print(classification_report(labels, predicted, zero_division=1))
print(classification_report(t2, t3, zero_division=1))

              precision    recall  f1-score   support

           0       1.00      0.00      0.00      1000
           1       1.00      0.00      0.00      1000
           2       1.00      0.00      0.00      1000
           3       0.19      0.11      0.14      1000
           4       0.00      0.00      0.00      1000
           5       1.00      0.00      0.00      1000
           6       0.10      0.06      0.07      1000
           7       0.00      0.00      0.00      1000
           8       0.03      0.09      0.05      1000
           9       0.17      1.00      0.28      1000

    accuracy                           0.13     10000
   macro avg       0.45      0.13      0.05     10000
weighted avg       0.45      0.13      0.05     10000



In [12]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(t2,t3))

[[   0    0    0    1    6    0  355    1  318  319]
 [   0    0    0  470   49    0   20    0   97  364]
 [   0    0    0    0    0    0   20    0  614  366]
 [   0    0    0  110   53    0  107    0  291  439]
 [   0    0    0    0    0    0   14    0  792  194]
 [   0    0    0    0    0    0    0    0    0 1000]
 [   0    0    0    3    1    0   56    0  498  442]
 [   0    0    0    0    0    0    0    0    0 1000]
 [   0    0    0    0    0    0    1    0   90  909]
 [   0    0    0    0    0    0    0    0    1  999]]
